# Build context

During image building, Docker can only access files from the build context. The path to the folder that specifies the build context is only a positional argument of the `docker build` command. 

## Check files

The only option to list files acceptalbe in the context is to build the image based on the docker file specified in the following cell in the context.

In [94]:
cat << EOF > dockerfile
FROM alpine
COPY . /context/
EOF

This docker file simply compiles the full context (`.`) into the container's `context` folder.

In the following example we create folder that we'll use as context and generate some random files tree.

In [95]:
mkdir example_context

# Function to create a random string
random_string() {
    cat /dev/urandom | tr -cd 'a-z0-9' | head -c 8
}

OBJECTS_NUMBER=10
curr_dir="example_context"

for ((i=0; i<OBJECTS_NUMBER; i++)) do
    object_name="$curr_dir/$(random_string)"

    if [ $(($RANDOM % 4)) -eq 0 ]; then
        mkdir $object_name
        curr_dir=$object_name
    else
        echo $(fortune) > $object_name
    fi
done

tree example_context

example_context
├── 1twdr3sj
├── bo24x8fs
└── dp285kht
    ├── 897im87i
    ├── a7iq06y7
    └── twrr7x9z
        ├── 914fsggq
        ├── kjj9eq99
        ├── rpjs7mik
        └── y6c6kn2n

2 directories, 8 files


Now try to build the image, run the container and check the `tree` of the context folder.

In [98]:
docker build -f ./dockerfile -t print_context ./example_context/ &> /dev/null
docker run --rm print_context tree context

context
├── 1twdr3sj
├── bo24x8fs
└── dp285kht
    ├── 897im87i
    ├── a7iq06y7
    └── twrr7x9z
        ├── 914fsggq
        ├── kjj9eq99
        ├── rpjs7mik
        └── y6c6kn2n

2 directories, 8 files


So it's identical to the context path tree on the host.

Now let us keep the environment clean by deleting everything we created in this example. 

In [99]:
docker rmi print_context
rm -r example_context
rm dockerfile

Untagged: print_context:latest
Deleted: sha256:d771db2a28f5efd197756ca6d204337447ba1c4c752550d5cd786def87a1a24c
